# Home assignment 2

You should work on the assignement in groups of 2 participants. 

Upload your solution as a jupyter notebook to L2P by 26th of June 23:59h. (The deadline is strict)

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results.

# 1. Dynamic PageRank
Consider a random walk setting where the transistion matrix changes over time. At any point of time the probability of a random surfer to jump to a linked page is proportional to the number of previous visits. To start with all the pages are equally likely to be chosen but as the walk continues and the nodes are visited the transition probability changes as proportional to number of previous visits. For example let a page 'a' is linked to pages 'b', 'c' and 'd'. The random surfer currently resides at 'a' and the pages 'b', 'c' and 'd' have already been visited 5, 3 and 2 times respectively. The transition probability would be 0.5, 0.3 and 0.2 respectively. As a new node is viited the probabilities change. The random surfer continues to surf with probability 0.8. Generate 100 random walks and rank the nodes based on the frequency of visit. The random walk should be performed on a drected Erdos-Renyi graph with number of nodes n=200 and probability of edge creation p = 0.4. 

Hint: Use networkx library for generating graph.
    

# 2. Recommendation
a. Compare the recommendation algorithms (SVD, NMF, Baseline, k-NN and Random) available in surprise package on movielens dataset in terms of RMSE and MAE.

b. Consider the movielens dataset and divide it into (i) training set with 50% of the data (train the algorithms on this part) and (ii) 25% validation set and (iii) test set with the rest. Estimate the ratings of the test set using the algorithms (same as in a) provided by the package on the training set. Your final rating should be weighted average of the ratings predicted by the algorithms. The weights should be learnt on the validation set. Performance should be measured in terms of RMSE.

Hint: Use grid search/step-wise update like SGD for learning the weights

# 3. Hidden Markov model
Consider the HMM package https://hmmlearn.readthedocs.io/en/latest/

a. Generate sequences with multinomial HMM (2 symbols and 4 hidden states) and given parameters. Start probability - {0.4,0.2,0.1,0.3}, Transition matrix - {{0.2,0.3,0.1,0.4},{0.3,0.3,0.2,0.2},{0.4,0.2,0.3,0.1},{0.2,0.3,0.1,0.4}}, Emission probability - {{0.2,0.8},{0.1,0.9},{0.5,0.5},{0.6,0.4}}.


b. Consider a sequence - {1 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1}. Fit a multinomial HMM considering 4 states and obtain hidden state which is most likely to have generated the symbol


# 4. PrefixSpan 
Implement the Prefix algorithm for Sequential Pattern Mining.